# Add Maxwell-Boltzmann Thermal Disorder to a Material

Add Maxwell-Boltzmann thermal disorder to a material by applying random displacements to atomic positions based on temperature. The displacement variance for each atom is proportional to kT/m, where k is the Boltzmann constant, T is the temperature, and m is the atomic mass.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click "Run" > "Run All" to run all cells. 
1. Wait for the run to complete (depending on the parameters can take a few min). 
1. Scroll down to view results. 

## Notes

1. For more information, see [Introduction](Introduction.ipynb)
1. The disorder parameter is typically the temperature in Kelvin for Maxwell-Boltzmann distribution.
1. Each atom's displacement variance is mass-dependent by default (kT/m).
1. Setting a random seed ensures reproducible results.

## 1. Prepare the Environment
### 1.1. Set up Maxwell disorder parameters

In [ ]:
# Temperature in Kelvin for Maxwell-Boltzmann distribution
TEMPERATURE_K = 300.0

# Random seed for reproducibility (set to None for random behavior)
RANDOM_SEED = 42

# Whether to use mass-dependent displacement (kT/m) or mass-independent (kT)
USE_MASS_DEPENDENT_DISPLACEMENT = True

# Optional: Create a supercell before applying disorder
CREATE_SUPERCELL = False
SUPERCELL_MATRIX = [[4, 0, 0], [0, 4, 0], [0, 0, 1]]

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("")

### 1.3. Get input materials
Materials are loaded with `get_materials()`.

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

### 1.4. Create and preview Supercell (Optional)
Creating a supercell before applying disorder can help visualize the effect better.

In [ ]:
from utils.visualize import visualize_materials as visualize
from mat3ra.made.tools.helpers import create_supercell

unit_cell = materials[0]

if CREATE_SUPERCELL:
    material = create_supercell(unit_cell, supercell_matrix=SUPERCELL_MATRIX)
    print(f"Created supercell with {len(material.basis.coordinates.values)} atoms")
else:
    material = unit_cell
    print(f"Using unit cell with {len(material.basis.coordinates.values)} atoms")

visualize(material, repetitions=[1, 1, 1], rotation="0x")

## 2. Create Maxwell-Boltzmann Disorder
### 2.1. Apply Maxwell-Boltzmann displacement
Use the Maxwell-Boltzmann displacement function to add thermal disorder to the material.

In [ ]:
from mat3ra.made.tools.build_components.operations.core.modifications.perturb.functions.maxwell_boltzmann import (
    create_maxwell_displacement_function,
)
from mat3ra.made.tools.operations.core.unary import perturb

# Create the Maxwell-Boltzmann displacement function
displacement_function = create_maxwell_displacement_function(
    material=material,
    disorder_parameter=TEMPERATURE_K,
    random_seed=RANDOM_SEED,
    is_mass_used=USE_MASS_DEPENDENT_DISPLACEMENT,
)

# Apply the displacement to create the disordered material
material_with_disorder = perturb(
    material=material,
    perturbation_function=displacement_function,
    use_cartesian_coordinates=True,
)

print(f"Applied Maxwell-Boltzmann disorder at T={TEMPERATURE_K}K")
print(f"Number of atoms: {len(material_with_disorder.basis.coordinates.values)}")

## 3. Visualize the Result
Compare the original material with the material after applying thermal disorder.

In [ ]:
visualize([
    {"material": material, "title": "Original material"},
    {"material": material_with_disorder, "title": f"Material with Maxwell disorder (T={TEMPERATURE_K}K)"},
    {"material": material_with_disorder, "title": f"Material with Maxwell disorder (side view)", "rotation": "-90x"},
])

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials

set_materials(material_with_disorder)